<a href="https://colab.research.google.com/github/griffinsnest/Midterm-Review/blob/main/population%3Dtax-data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn import linear_model as lm


# Load data from CSV files into variable arrays
taxData = pd.read_csv('/content/sales-and-use-tax (1).csv')
popData = pd.read_csv('/content/HS-STAT-Population-of-Vermont-towns-1930-2019 (1).csv')

# Data Eng. Steps
# drop the column saving IDs from data set as it isn't needed
popData = popData.drop('CTC', axis = 1)

# Reshape data
popData2 = popData.melt(id_vars=['NAME'], var_name='Calendar Year', value_name='Population')
# Make sure data we're joining on are matching data types
popData2 = popData2.astype({'Calendar Year' : 'int64', 'Population' : 'int32'})
# Merge Taxdata with po data two, left is the what on the left we are joinning, right the right side being joined,
# and inner makes it drop all none matching joins
DATA = taxData.merge(popData2, left_on=['Town', 'Calendar Year' ], right_on=['NAME', 'Calendar Year'], how='inner')


#DATA Mining Steps
TargetFeature = "Gross"
DATAFeatures = DATA.drop(TargetFeature, axis=1)
MostValuedFeature = DATAFeatures.std().sort_values(ascending=False)[0:1]
LeastValuedFeature = DATAFeatures.std().sort_values()[0:1]

#Testing Data Mining Findings
# Fill X vector with data from "Retail" column for linear regression 
# as it had the highest STD
# X variables use DataFeatures while Y utilize original DATA
xb = np.array(DATAFeatures[MostValuedFeature.index[0]])
xb = xb.reshape(-1,1)

# Fill Y vector with data from "Gross" column for linear regression
yb = np.array(DATA[TargetFeature])
yb = yb.reshape(-1,1)

# feed both vectors into linear regression and save score of it into var.
bestModel = lm.LinearRegression().fit(xb, yb)
BMScore = bestModel.score(xb, yb)

# Fill X vector with data from "Calendar Year" column for linear regression
xw = np.array(DATAFeatures[LeastValuedFeature.index[0]])
xw = xw.reshape(-1,1)
# Fill Y vector with data from "Gross" column for linear regression
yw = np.array(DATA[TargetFeature])
yw = yw.reshape(-1,1)

# feed both vectors into linear regression and save score of it into var.
worstModel = lm.LinearRegression().fit(xw, yw)
WMScore = worstModel.score(xw, yw)

#New Data Model with best Features
bestDataModel = DATA[[TargetFeature, MostValuedFeature.index[0]]]
print(bestDataModel.head())

# print data findings
print('Best Model Score:', BMScore)
print('Worst Model Score:', WMScore)


          Gross       Retail
0  1.831730e+06    489006.65
1  3.246572e+07   1857137.48
2  2.203010e+08  14313220.56
3  6.075238e+07  16720375.88
4  5.708651e+08  93688630.25
Best Model Score: 0.8663761236994375
Worst Model Score: 0.00014268782597559593


In [ ]:
from google.colab import drive
drive.mount('/content/drive')